# Business Problem

The Tanzania Development Trust is a UK charitable organization operating within the country of Tanzania since 1975.

They focus on development in rural Tanzania, aiming to support small projects in the poorest parts of the country where one of their priority areas of funding is clean water. Their stated water project involves boreholing and rope pump installation in areas with limited access to clean water, currently located in the regions of Kagera and Kigoma in the northwest of the country.

A new benefactor wants to expand the project not only geographically to more of the country, but in the scope of repairing existing pumps before they fail. I have been tasked with developing a model to predict the operating condition of a current waterpoint: functional, needs repair, or non-functional.

The main objective is to identify waterpoints that are in need of repair. [Research shows](https://sswm.info/entrepreneurship-resource/developing-impactful-businesses/maintenance-services-for-rural-water-pumps) that it is much less expensive to repair and rehabilitate a waterpoint, as well as being more protective of the water resources in the country. 
The secondary objective is to identify concentrations of non-functioning water points that may be an eligible location for a new installation.
______________________
The data provided for modeling was collected between March 2011 and March 2013, and contains the information for 59,400 water points 

# Imports

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# import training data and target
raw_data = pd.read_csv('data/training_data.csv')
raw_target = pd.read_csv('data/training_target.csv')

display(raw_target.info())
print(raw_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            59400 non-null  int64 
 1   status_group  59400 non-null  object
dtypes: int64(1), object(1)
memory usage: 928.2+ KB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     59400 non-null  int64  
 1   amount_tsh             59400 non-null  float64
 2   date_recorded          59400 non-null  object 
 3   funder                 55765 non-null  object 
 4   gps_height             59400 non-null  int64  
 5   installer              55745 non-null  object 
 6   longitude              59400 non-null  float64
 7   latitude               59400 non-null  float64
 8   wpt_name               59400 non-null  object 
 9   num_private            59400 non-null  int64  
 10  basin                  59400 non-null  object 
 11  subvillage             59029 non-null  object 
 12  region                 59400 non-null  object 
 13  region_code            59400 non-null  int64  
 14  district_code          59400 non-null  int64  
 15  lg

Create a raw dataframe with merged data and target. We will use this df during initial EDA so we can compare feature relationships with target, and so we can understand and deal with null values

In [3]:
raw_df = pd.merge(raw_data, raw_target, on='id')

The dataset for training includes 59,400 entries with 39 total features, a unique identifier, and a target label.

In [4]:
status_values = pd.DataFrame(raw_df.status_group.value_counts())
status_values['percent'] = round(raw_df.status_group.value_counts(normalize=True) * 100, 1)
status_values

,status_group,percent
functional,32259,54.3
non functional,22824,38.4
functional needs repair,4317,7.3


This is a ternary classification problem. The three possible values are:
- functional (F)
- non functional (NF)
- functional needs repair (FR)

Value counts show that our dataset is not balanced with respect to the label values. Only 7.3% of pumps are classified as functional needs repair, while 54.3% are functional and 38.4% are non functional. We will need to keep this imbalance in mind when modeling.
_________
Before any modeling can occur we must check and deal with null values


# Null Checks

In [5]:
null_checks = pd.DataFrame(data=raw_df.isna().sum(),
                          columns=['null_count'])
null_checks['percent_of_data'] = round((null_checks.null_count / len(raw_data)) * 100, 1)
null_checks = null_checks[null_checks.percent_of_data > 0.1]
null_checks.sort_values('percent_of_data', ascending=False, inplace=True)
null_checks

,null_count,percent_of_data
scheme_name,28166,47.4
scheme_management,3877,6.5
installer,3655,6.2
funder,3635,6.1
public_meeting,3334,5.6
permit,3056,5.1
subvillage,371,0.6


There are 7 features with null values in our dataset, and we can what that number of nulls is by percent of total available data. 
______
All of the features that contain null values are object types and will need to be converted. Before conversion we will need to address the null values.

## subvillage

In [6]:
subvillage_nans = raw_df[raw_df.subvillage.isnull()]
round(subvillage_nans.status_group.value_counts(normalize=True) * 100, 2)

functional                 55.26
non functional             44.47
functional needs repair     0.27
Name: status_group, dtype: float64

The null values in subvillage represent 0.6% of our total data. The distribution of the target label is close to the whole dataset

In [7]:
subvillage_nans.region.value_counts()

Dodoma    361
Mwanza     10
Name: region, dtype: int64

All but 10 of our subvillage nan's come from the region of Dodoma, the rest from Mwanza. Lets look at the subvillage distribution of those regions from the whole dataset

In [8]:
raw_df[raw_df['region'] == 'Dodoma'].subvillage.value_counts()

Kawawa         54
Shuleni        43
Nyerere        35
Azimio         34
Majengo        32
               ..
Foye            1
Mtatangwe       1
Makao Mapya     1
Soya Mjini      1
Mgomwa          1
Name: subvillage, Length: 705, dtype: int64

In [9]:
raw_df[raw_df['region'] == 'Mwanza'].subvillage.value_counts()

1                     132
Madukani               52
Bujingwa               25
Shuleni                19
Matale                 18
                     ... 
Bukalo                  1
Nyambona                1
Kabaganda B             1
Bulyahilu Center B      1
Mwambogwa               1
Name: subvillage, Length: 1507, dtype: int64

There are no average or overwhelmingly dominant subvillages that we could assign the null values to. It's not clear if we will use subvillage in modeling, so we will change null values to 'Other'

In [10]:
raw_df['subvillage'].fillna(value='Other', inplace=True)

## permit

In [11]:
permit_nans = raw_df[raw_df.permit.isnull()]
permit_nans.reset_index(drop=True, inplace=True)
round(permit_nans.status_group.value_counts(normalize=True) * 100, 2)

functional                 54.74
non functional             35.44
functional needs repair     9.82
Name: status_group, dtype: float64

5% of our dataset have no value for permit. Distribution of the target label is approximately the same as the whole dataset.

In [12]:
permit_distribution = raw_df.permit.value_counts(normalize=True)
permit_distribution

True     0.68955
False    0.31045
Name: permit, dtype: float64

Per the data documentation, the permit feature is if the water point is permitted or not. Data we do have for this feature show it's about 70/30 in favor of permitted.

We will randomly fill these 3056 missing datapoints with true/false in the same ratio we found in our entire dataset.

In [13]:
raw_df['permit'] = raw_df['permit'].fillna(pd.Series(np.random.choice([True, False],
                                                       p=list(permit_distribution),
                                                       size=len(raw_df))))

## public_meeting

In [14]:
public_meeting_nans = raw_df[raw_df.public_meeting.isnull()]
public_meeting_nans.reset_index(drop=True, inplace=True)
round(public_meeting_nans.status_group.value_counts(normalize=True) * 100, 2)

functional                 50.33
non functional             44.99
functional needs repair     4.68
Name: status_group, dtype: float64

5.6% of our dataset has no value for public_meeting. Distribution of the target label is approximately the same as the whole dataset.

In [15]:
meeting_distribution = raw_df.public_meeting.value_counts(normalize=True)
meeting_distribution

True     0.909838
False    0.090162
Name: public_meeting, dtype: float64

The public meeting feature is a boolean that is over 90% true for data we do have. We will fill null values in the same percentages.

In [16]:
raw_df['public_meeting'] = raw_df['public_meeting'].fillna(
    pd.Series(np.random.choice([True, False],
                               p=list(meeting_distribution),
                               size=len(raw_df))))

## funder & installer

The features 'funder' and 'installer' have almost the same number of null values; I am curious about the overlap of nulls.

In [17]:
# dividing the number of entries with null for both features by the smaller count
len(raw_df[raw_df.funder.isnull() & raw_df.installer.isnull()]) / null_checks.null_count['funder']

0.9854195323246218

Over 98% of the null values for funder also contain null values for installer.
________

In [18]:
funder_df = pd.DataFrame(round(raw_df.funder.value_counts(normalize=True, dropna=False) * 100, 2))
funder_df

,funder
Government Of Tanzania,15.29
NaN,6.12
Danida,5.24
Hesawa,3.71
Rwssp,2.31
...,...
Rarymond Ekura,0.00
Justine Marwa,0.00
Municipal Council,0.00
Afdp,0.00


In [19]:
funder_df_top = funder_df[funder_df.funder > 1.0]
print(f"Funders with more than 1% share: {len(funder_df_top)}")
print(f"Percent of total funders represented by above: {funder_df_top.sum()}")

Funders with more than 1% share: 18
Percent of total funders represented by above: funder    52.69
dtype: float64


Including null values, there were 1,898 distinct values for funder. Of that, 18 values (including null) have representative counts more than 1% of total data.

Those 18 distinct values represent almost 53% of our total data. We will convert null values to 'Other'. There are still lots of unique values, so something to consider is converting all funders with less than 1% total share as 'Other' to reduce the unique value count.

In [20]:
installer_df = pd.DataFrame(round(raw_df.installer.value_counts(normalize=True, dropna=False) * 100, 2))
installer_df

,installer
DWE,29.30
NaN,6.15
Government,3.07
RWE,2.03
Commu,1.78
...,...
Wizara ya maji,0.00
TWESS,0.00
Nasan workers,0.00
R,0.00


In [21]:
installer_df_top = installer_df[installer_df.installer > 1.0]
print(f"Installers with more than 1% share: {len(installer_df_top)}")
print(f"Percent of total installers represented by above: {installer_df_top.sum()}")

Installers with more than 1% share: 12
Percent of total installers represented by above: installer    51.6
dtype: float64


Similarly to funder, the installer feature is dominated by small share installers. Of the 2,146 distinct values for installer, 12 values (including null) have representative counts more than 1% of total data.

Those 12 distinct values represent almost 52% of our total data. This is similar to the funder feature. We will also convert null values to 'Other', and will consider converting all installers with less than 1% total share of installer as 'Other' to reduce the unique value count. 

In [22]:
raw_df['funder'].fillna(value='Other', inplace=True)
raw_df['installer'].fillna(value='Other', inplace=True)

## scheme_management

In [23]:
scheme_management_nans = raw_df[raw_df.scheme_management.isnull()]
scheme_management_nans.reset_index(drop=True, inplace=True)
round(scheme_management_nans.status_group.value_counts(normalize=True) * 100, 2)

functional                 48.31
non functional             45.94
functional needs repair     5.75
Name: status_group, dtype: float64

In [24]:
raw_df.scheme_management.value_counts(normalize=True, dropna=False)

VWC                 0.619411
WUG                 0.087643
NaN                 0.065269
Water authority     0.053081
WUA                 0.048535
Water Board         0.046263
Parastatal          0.028283
Private operator    0.017896
Company             0.017862
Other               0.012896
SWC                 0.001633
Trust               0.001212
None                0.000017
Name: scheme_management, dtype: float64

6.5% of our data has no value for scheme_management. Distribution of the target data is approximately the same as the whole dataset.
_________
There was only one entry with the value of 'None', we will change that to 'Other'

In [25]:
raw_df.at[23603, 'scheme_management'] = 'Other'

We will fill null values for scheme_management randomly with other values from the feature in the same percentage.

In [26]:
scheme_management_list = pd.DataFrame(raw_df.scheme_management.value_counts(normalize=True))
scheme_management_list

,scheme_management
VWC,0.662662
WUG,0.093763
Water authority,0.056787
WUA,0.051924
Water Board,0.049493
Parastatal,0.030258
Private operator,0.019145
Company,0.019109
Other,0.013814
SWC,0.001747


In [27]:
raw_df['scheme_management'] = raw_df['scheme_management'].fillna(
    pd.Series(np.random.choice(list(scheme_management_list.index),
                               p=list(scheme_management_list.scheme_management),
                               size=len(raw_df))))

## scheme_name

In [28]:
raw_df.scheme_name.value_counts(normalize=True, dropna=False)

NaN                        0.474175
K                          0.011481
None                       0.010842
Borehole                   0.009192
Chalinze wate              0.006818
                             ...   
Visiga water supplly       0.000017
Emanyata pipelines         0.000017
Magundi water supply       0.000017
Imalampaka water supply    0.000017
Mtawanya                   0.000017
Name: scheme_name, Length: 2697, dtype: float64

Almost half (47%) of the scheme_name feature contains no data, and the remaining data contains 2,697 distinct other features, none of which exceed 1% of the dataset. Lets look a little closer to see what else we can figure out.

In [29]:
scheme_name = pd.DataFrame(raw_df.scheme_name.value_counts(dropna=False))
scheme_name[scheme_name.scheme_name > 75]

,scheme_name
NaN,28166
K,682
None,644
Borehole,546
Chalinze wate,405
M,400
DANIDA,379
Government,320
Ngana water supplied scheme,270
wanging'ombe water supply s,261


The scheme name, per the documentation, is the individual or group that actually operates the waterpoint. This is compared to the scheme management company, which oversees operation. When the data was collected, it looks like there was little organization with respect to this particular datapoint. Considering the there are no other patterns in the available names, and that we have the management data, we will likely not use this feature in modeling.
_____
We will replace null values with 'None'

In [30]:
raw_df['scheme_name'].fillna(value='None', inplace=True)

# Exploring numerical data

Creating a dataframe of just the current numerical features as well as the target. Mapping numerical values to the target: 'non functional' = 0, 'functional needs repair' = 1, and 'functional' = 2.

In [31]:
num_df = pd.merge(raw_data.select_dtypes(include=np.number), raw_target, on='id')

status_dict = {'non functional': 0,
              'functional needs repair': 1,
              'functional': 2}

num_df['target'] = num_df['status_group'].map(lambda x: status_dict[x])
num_df.drop('status_group', axis=1, inplace=True)
num_df.describe()

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year,target
count,59400.000000,59400.000000,59400.000000,59400.000000,5.940000e+04,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000
mean,37115.131768,317.650385,668.297239,34.077427,-5.706033e+00,0.474141,15.297003,5.629747,179.909983,1300.652475,1.158838
std,21453.128371,2997.574558,693.116350,6.567432,2.946019e+00,12.236230,17.587406,9.633649,471.482176,951.620547,0.949794
min,0.000000,0.000000,-90.000000,0.000000,-1.164944e+01,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,18519.750000,0.000000,0.000000,33.090347,-8.540621e+00,0.000000,5.000000,2.000000,0.000000,0.000000,0.000000
50%,37061.500000,0.000000,369.000000,34.908743,-5.021597e+00,0.000000,12.000000,3.000000,25.000000,1986.000000,2.000000
75%,55656.500000,20.000000,1319.250000,37.178387,-3.326156e+00,0.000000,17.000000,5.000000,215.000000,2004.000000,2.000000
max,74247.000000,350000.000000,2770.000000,40.345193,-2.000000e-08,1776.000000,99.000000,80.000000,30500.000000,2013.000000,2.000000


The 'id' feature is a unique identifier, we will leave it in for merge purposes later but will not use it in modeling so no investigation needed.

The 'amount_tsh' feature is the total amount of water available to the waterpoint. It looks like at least 50% of our waterpoints do not have any water available, regardless of pump functionality. If we look back, we also see that over 50% of pumps are classified as functional. This is a bit concerning, it's unclear how you can have a functional pump with no water available to pump.

'gps_height' is concerning. [Research shows](https://en.wikipedia.org/wiki/Geography_of_Tanzania) that the lowest point in the country is sea level (0), yet we have a minimum of -90, so we will need to investigate that.

[Research shows](https://worldpopulationreview.com/countries/tanzania/location) that Tanzania's most extreme latitudes range from 00&deg;59'S (-0.98333) to 11&deg;45'S (-11.75), while the longitude extremes range from 40&deg;29'E (40.48333) to 29&deg;10'E (29.16667)

The 'latitude' values seem to exceed the northen border of the country (max latitude -.00000002), so we will need to investigate that. The 'longitude' values minimum is 0, so we have some data that is outside the range of the country borders. It's likely from mistakens in data entry, but we need to investigate the extent and fix it.

There is no description of 'num_private' in the datasource, we'll have to look at that. It may not factor into modeling.

'region_code' and 'district_code' will need to be investigated separately as well, the documenation is not clear about what that data represents.

'construction_year' and 'population' each have minimums of 0, and that should defintely not be the case in construction year. It's possible a waterpoint exists in a place considered to have no population, but we'll need to investigate these further.

## amount_tsh

In [32]:
len(num_df[num_df.amount_tsh == 0.0]) / len(num_df)

0.700993265993266

70% of our datapoints list the amount_tsh as equal to 0.0. This can't mean that the waterpoint has no water available. We need to understand more about what total static head means. [Research shows](https://www.pumpfundamentals.com/tutorial2.htm#:~:text=If%20the%20liquid%20surface%20of,the%20friction%20in%20the%20system.) that a pump system's static head is the difference between the liquid surface of the reservoir and the discharge end of the pump system. The higher the discharge tube is lifted above the liquid surface, the harder is is for the pump to move the water, and the lower the flow rate will be. I would imagine that the requirements of pump location to discharge would impact the type of pump to be used.

In [33]:
num_df.groupby('target').mean()[['amount_tsh']]

,amount_tsh
target,
0,123.481230
1,267.071577
2,461.798235


In [34]:
num_df[num_df.amount_tsh > 0]

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year,target
0,69572,6000.0,1390,34.938093,-9.856322,0,11,5,109,1999,2
2,34310,25.0,686,37.460664,-3.821329,0,21,4,250,2009,2
5,9944,20.0,0,39.172796,-4.765587,0,4,8,1,2009,2
11,50409,200.0,1062,35.770258,-10.574175,0,10,5,250,1987,2
16,48451,500.0,1703,34.642439,-9.106185,0,11,4,35,1978,0
...,...,...,...,...,...,...,...,...,...,...,...
59385,34473,500.0,1327,33.951681,-2.021854,0,20,4,200,2011,2
59387,26640,100.0,25,39.176480,-6.957098,0,7,2,100,2000,2
59394,11164,500.0,351,37.634053,-6.124830,0,5,6,89,2007,0
59395,60739,10.0,1210,37.169807,-3.253847,0,3,5,125,1999,2


The average total static head increases with each pump status category, from 0 (non function) to 2 (functional). We will have to engineer this feature in some way in order to utilize it. Most of our values are 0, we have none that are below that, and the maximum value is 350,000 feet.

## gps_height

In [35]:
gps_height_neg = num_df[num_df.gps_height < 0]
print(f"Percent of total data: {round(len(gps_height_neg)/len(num_df)*100,2)}%")
gps_height_neg.describe()

Percent of total data: 2.52%


,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year,target
count,1496.000000,1496.000000,1496.000000,1496.000000,1496.000000,1496.000000,1496.000000,1496.000000,1496.000000,1496.000000,1496.000000
mean,37297.606952,313.013369,-19.993316,39.352801,-7.727535,0.506684,34.703877,15.531417,258.137032,1686.096257,1.073529
std,21119.500129,4017.641868,12.154136,0.458811,1.542230,7.155227,35.259116,21.812220,428.417873,726.379806,0.983446
min,150.000000,0.000000,-90.000000,38.614960,-10.946096,0.000000,4.000000,1.000000,1.000000,0.000000,0.000000
25%,19232.750000,0.000000,-28.000000,38.972421,-8.584131,0.000000,6.000000,1.000000,40.000000,1976.000000,0.000000
50%,36699.000000,0.000000,-18.000000,39.281546,-7.415977,0.000000,7.000000,5.000000,102.500000,2000.000000,2.000000
75%,55492.750000,50.000000,-11.000000,39.662349,-6.524902,0.000000,60.000000,13.000000,320.000000,2008.000000,2.000000
max,74211.000000,138000.000000,-1.000000,40.345193,-5.278598,150.000000,99.000000,67.000000,4520.000000,2013.000000,2.000000


There are 1,496 datapoints where the gps_height was less than 0, about 2.5% of our data. Earlier we mentioned that this should not be possible as the listed lowest elevation for Tanzania is 0 ft above sea level (at the Indian Ocean). [Research shows](https://eos-gnss.com/knowledge-base/articles/elevation-for-beginners) that most GPS units are designed to measure height based on the representation of the earth's surface as an 'ellipsoid', and it's perfectly possible to be standing at sea level where the reading should be 0 and have it be a negative number. 

Considering these data are almost all on the eastern edge of the country (lower longitudes) closer to the Indian Ocean, it is likely the sites are at or just above sea level and capable of producing negative height readings. Opportunity for further tuning could be to address this incosistency across all the data, shifting all datapoints to a more accurage representation of height. I do not think it would be as simple as shifting all values up by the largest negative difference. Because it's such a small percentage of our dataset, we will just set them to 0 rather than worry about adjusting all the values.

In [36]:
num_df.gps_height.clip(lower=0.0, inplace=True)

## longitude/latitude

The max longitude is within the boundaries, we need to examine all values below the actual minimum which is 29.16667.

In [37]:
longitude_errors = num_df[num_df.longitude < 29.16667]
longitude_errors.describe()

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year,target
count,1812.000000,1812.0,1812.0,1812.0,1.812000e+03,1812.0,1812.000000,1812.000000,1812.0,1812.0,1812.000000
mean,37389.841060,0.0,0.0,0.0,-2.000000e-08,0.0,17.820088,2.497241,0.0,0.0,1.173289
std,21413.129962,0.0,0.0,0.0,3.309636e-24,0.0,1.023562,2.157389,0.0,0.0,0.870267
min,15.000000,0.0,0.0,0.0,-2.000000e-08,0.0,11.000000,1.000000,0.0,0.0,0.000000
25%,18481.750000,0.0,0.0,0.0,-2.000000e-08,0.0,17.000000,1.000000,0.0,0.0,0.000000
50%,37326.000000,0.0,0.0,0.0,-2.000000e-08,0.0,17.000000,1.000000,0.0,0.0,1.000000
75%,55509.750000,0.0,0.0,0.0,-2.000000e-08,0.0,19.000000,6.000000,0.0,0.0,2.000000
max,74193.000000,0.0,0.0,0.0,-2.000000e-08,0.0,19.000000,6.000000,0.0,0.0,2.000000


In [38]:
print(f"Percent of data with missing lat/long: {round((len(longitude_errors)/len(num_df))*100, 2)}%")

print(f"Total data with population of 0: {len(num_df[num_df.population == 0])}")
print(f"Total data with construction year of 0: {len(num_df[num_df.construction_year == 0])}")

Percent of data with missing lat/long: 3.05%
Total data with population of 0: 21381
Total data with construction year of 0: 20709


It looks like we have found that our data includes errors in GPS readings. These errors show up as longitude 0 and latitude -2e-0.8. These entries with GPS errors, however, do not account for all the population point values of 0, or the construction year values of 0, so we will still need to address them.

In [48]:
gps_errors = raw_df[raw_df.longitude == 0.0]
gps_errors.status_group.value_counts(normalize=True)

functional                 0.480132
non functional             0.306843
functional needs repair    0.213024
Name: status_group, dtype: float64

The missing GPS data contains a disproportionate share of the functional but needs repair. As we investigate the region and district codes we can hopefully gain insight to the relationship between them, i.e. what is the parent/child relationship, and what children are under what parents. What we can then do is establish generic gps coordinates to the specific region/district code as the average of the gps coordinates we do have for that area, and replace our null values with that average (with a little randomness).

## num_private

In [52]:
num_df.num_private.value_counts(normalize=True)

0       0.987256
6       0.001364
1       0.001229
5       0.000774
8       0.000774
          ...   
42      0.000017
23      0.000017
136     0.000017
698     0.000017
1402    0.000017
Name: num_private, Length: 65, dtype: float64

The vast majority of the num_private data has the value 0. There was no descriptor of the feature along with the others for the dataset, so it's likely that this feature won't be used in modeling. We don't have any null values to convert.

## region/district_code

What we suspect is that the regions each have multiple districts in them.

In [130]:
print(f"Region numeric unique value count: {len(num_df.region_code.value_counts())}")
print(f"Region string unique value count: {len(raw_df.region.value_counts())}")

Region numeric unique value count: 27
Region string unique value count: 21


The unique numeric value count for region exceed that for the string encoded region.

In [131]:
region_df = pd.DataFrame(raw_df.region.value_counts())
region_df.reset_index(inplace=True)
region_df.columns = ['region', 'str_count']
# adding in the top 21 values from region_code, since the region_code
# has more values than 'region'
region_df['region_code'] = raw_df.region_code.value_counts()[:21].index
region_df['code_count'] = raw_df.region_code.value_counts()[:21].values
region_df

,region,str_count,region_code,code_count
0,Iringa,5294,11,5300
1,Shinyanga,4982,17,5011
2,Mbeya,4639,12,4639
3,Kilimanjaro,4379,3,4379
4,Morogoro,4006,5,4040
5,Arusha,3350,18,3324
6,Kagera,3316,19,3047
7,Mwanza,3102,2,3024
8,Kigoma,2816,16,2816
9,Ruvuma,2640,10,2640


When we match up the values of the region and region_code features by their respective value counts we can see that some of them are very close, if not equal. 
__________

These are the bottom 6 remaining region_code value counts that did not match up with the quantity of values found in the text string. 

In [132]:
spare_region_codes = pd.DataFrame(raw_df.region_code.value_counts()[21:])
spare_region_codes.reset_index(inplace=True)
spare_region_codes.columns = ['region_code', 'code_count']
spare_region_codes

,region_code,code_count
0,7,805
1,99,423
2,9,390
3,24,326
4,8,300
5,40,1


In [133]:
raw_df[raw_df.region_code == 90].describe()

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
count,917.000000,917.000000,917.000000,917.000000,917.000000,917.000000,917.0,917.000000,917.000000,917.000000
mean,36718.627045,56.407852,375.494002,38.958680,-10.805449,0.494002,90.0,38.169029,173.992366,1875.147219
std,21869.242111,374.017393,129.407413,0.320121,0.192244,14.860311,0.0,11.335437,443.216263,469.481554
min,35.000000,0.000000,19.000000,38.013947,-11.366974,0.000000,90.0,33.000000,1.000000,0.000000
25%,17293.000000,0.000000,303.000000,38.684909,-10.922284,0.000000,90.0,33.000000,1.000000,1979.000000
50%,36231.000000,0.000000,352.000000,39.024774,-10.832262,0.000000,90.0,33.000000,44.000000,1988.000000
75%,56225.000000,20.000000,424.000000,39.192416,-10.658395,0.000000,90.0,33.000000,150.000000,2006.000000
max,74195.000000,5000.000000,879.000000,39.446226,-10.355590,450.000000,90.0,63.000000,5016.000000,2013.000000


Investigating the region code 90, we think is Dar es Salaam, but the GPS coordinates do not match (they are very far south). Lets look at region code 7. That value count is exactly the same as the one for the string coded Dar es Salaam but was cut off as we sorted by value count size.

In [134]:
raw_df[raw_df.region_code == 7].describe()

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
count,805.000000,805.000000,805.000000,805.000000,805.000000,805.0,805.0,805.000000,805.000000,805.000000
mean,37288.637267,236.706832,31.105590,39.215799,-6.909677,0.0,7.0,2.151553,240.843478,1958.782609
std,21473.569214,2612.250698,38.325135,0.130179,0.097479,0.0,0.0,0.600073,534.198421,288.091177
min,22.000000,0.000000,-63.000000,39.039088,-7.162921,0.0,7.0,1.000000,1.000000,0.000000
25%,17976.000000,0.000000,5.000000,39.117940,-6.974554,0.0,7.0,2.000000,90.000000,2000.000000
50%,37720.000000,50.000000,29.000000,39.174331,-6.908980,0.0,7.0,2.000000,150.000000,2000.000000
75%,56342.000000,50.000000,60.000000,39.335933,-6.878280,0.0,7.0,3.000000,250.000000,2010.000000
max,74111.000000,45000.000000,151.000000,39.537132,-6.576367,0.0,7.0,3.000000,9865.000000,2010.000000


When we look on Google earth at the mean values for the lat/long for the region code 7, it puts us right at Dar es Salaam. I think we can be confident that at least region 7 is Dar es Salaam. Now we can match up the rest.

In [135]:
# update the Dar es Salaam entry in the region dataframe with the correct region code
region_df.at[20,'region_code'] = 7
region_df.at[20, 'code_count'] = 805

# move the replaced region code and count into the spare region code dataframe
spare_region_codes.at[0,'region_code'] = 90
spare_region_codes.at[0,'code_count'] = 917

In [136]:
region_df[region_df.str_count == region_df.code_count]

,region,str_count,region_code,code_count
2,Mbeya,4639,12,4639
3,Kilimanjaro,4379,3,4379
8,Kigoma,2816,16,2816
9,Ruvuma,2640,10,2640
14,Mara,1969,20,1969
20,Dar es Salaam,805,7,805


6 of our 21 region string counts and code counts match up exactly. We will still verify

In [137]:
# helper function to get the .mean() lat/long values from the raw_df and
# create a tuple of them combined in a new feature
def get_lat_long(x):
    means = raw_df[raw_df.region_code == x][['latitude', 'longitude']].mean()
    return (means[0], means[1])

region_df['lat_long_avg'] = region_df.region_code.map(get_lat_long)
region_df

,region,str_count,region_code,code_count,lat_long_avg
0,Iringa,5294,11,5300,"(-8.899634086545284, 34.87527552078679)"
1,Shinyanga,4982,17,5011,"(-2.7549962549910196, 26.232338852508484)"
2,Mbeya,4639,12,4639,"(-9.097417654386721, 33.531504774080624)"
3,Kilimanjaro,4379,3,4379,"(-3.5228229253459693, 37.50510820328157)"
4,Morogoro,4006,5,4040,"(-7.40001297169802, 37.047041468495046)"
5,Arusha,3350,18,3324,"(-1.980647648850782, 31.25387806628761)"
6,Kagera,3316,19,3047,"(-1.9737615997899574, 24.924561809622578)"
7,Mwanza,3102,2,3024,"(-3.231962549540344, 36.64873768358135)"
8,Kigoma,2816,16,2816,"(-4.296219425625, 30.214676500465202)"
9,Ruvuma,2640,10,2640,"(-10.774378770549243, 35.72930405988637)"


In [108]:
from geopy import distance

In [139]:
test_lat_longs = {
    'Mbeya': (-8.909401, 33.460773),
    'Kilimanjaro': (-3.334883, 37.340382), # used lat/long for city of Moshi
    'Kigoma': (-4.893941, 29.673386),
    'Ruvuma' : (-10.676803, 35.655785), # used city of Songea
    'Mara' : (-1.510191, 33.802046), # used city of Musoma
    'Dar es Salaam' : (-6.776012, 39.178326)
}

In [140]:
loc = 'Dar es Salaam'

data_point = region_df.loc[20, 'lat_long_avg']
real_point = test_lat_longs[loc]

print(distance.distance(data_point, real_point).miles)

9.538862270734247


In [141]:
loc = 'Mara'

data_point = region_df.loc[14, 'lat_long_avg']
real_point = test_lat_longs[loc]

print(distance.distance(data_point, real_point).miles)

29.001829466285304


In [56]:
num_df.district_code.value_counts().sort_index()

0        23
1     12203
2     11173
3      9998
4      8999
5      4356
6      4074
7      3343
8      1043
13      391
23      293
30      995
33      874
43      505
53      745
60       63
62      109
63      195
67        6
80       12
Name: district_code, dtype: int64

In [44]:
len(num_df.region_code.value_counts())

27

21

In [40]:
gps_errors.region_code.value_counts()

17    1057
19     752
11       3
Name: region_code, dtype: int64

In [41]:
gps_errors.district_code.value_counts()

1    1057
6     488
2     264
4       3
Name: district_code, dtype: int64

In [42]:
gps_errors[(gps_errors.region_code == 17) & (gps_errors.district_code == 1)]

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
21,6091,0.0,2013-02-10,Dwsp,0,DWE,0.0,-2.000000e-08,Muungano,0,...,unknown,unknown,unknown,unknown,shallow well,shallow well,groundwater,hand pump,hand pump,functional
168,72678,0.0,2013-01-30,Wvt,0,WVT,0.0,-2.000000e-08,Wvt Tanzania,0,...,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
177,56725,0.0,2013-01-17,Netherlands,0,DWE,0.0,-2.000000e-08,Kikundi Cha Wakina Mama,0,...,soft,good,enough,enough,shallow well,shallow well,groundwater,other,other,non functional
321,45415,0.0,2013-01-22,Dwsp,0,DWE,0.0,-2.000000e-08,Upendo,0,...,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump,functional needs repair
323,51403,0.0,2013-02-16,Dwsp,0,DWE,0.0,-2.000000e-08,Serengeti,0,...,soft,good,enough,enough,other,other,unknown,other,other,non functional
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59189,62177,0.0,2011-07-18,Dwsp,0,DWE,0.0,-2.000000e-08,Wazazo,0,...,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump,functional needs repair
59208,3631,0.0,2013-01-22,Dwsp,0,DWE,0.0,-2.000000e-08,Mtakuja,0,...,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump,functional
59295,60843,0.0,2011-07-19,Rwssp,0,DWE,0.0,-2.000000e-08,Maendeleo,0,...,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump,functional needs repair
59324,748,0.0,2013-01-22,World Vision,0,World Vision,0.0,-2.000000e-08,Mwazwilo,0,...,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump,functional


# Exploring extraction types

In [ ]:
extraction_types = pd.DataFrame(raw_df.extraction_type.value_counts())
extraction_types.columns = ['total_count']
extraction_types['percent_of_total'] = round((extraction_types.total_count / len(raw_df)) * 100, 2)
extraction_types['non-functional'] = raw_df[raw_df.status_group == 'non functional'].extraction_type.value_counts()
extraction_types['non-functional_percent'] = round((extraction_types['non-functional'] / extraction_types.total_count) * 100, 2)
extraction_types['needs_repair'] = raw_df[raw_df.status_group == 'functional needs repair'].extraction_type.value_counts()
extraction_types.fillna(0.0, inplace=True)
extraction_types['needs_repair'] = extraction_types['needs_repair'].astype('int')
extraction_types['needs_repair_percent'] = round((extraction_types['needs_repair'] / extraction_types.total_count) * 100, 2)
extraction_types